# Modelo - Treino e Teste

**Importações**

In [37]:
import pandas as pd
pd.set_option('display.max_columns', None)

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
from collections import defaultdict

In [5]:
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",")
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",")
olist_order_payments = pd.read_csv("olist_order_payments_dataset.csv", sep=",")
olist_order_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",")
olist_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",")

In [6]:
dataframe = olist_customers.merge(olist_orders, on="customer_id") \
                           .merge(olist_order_payments, on="order_id") \
                           .merge(olist_order_reviews, on="order_id") \
                           .merge(olist_order_items, on="order_id")

**Ajustes**

In [17]:
df_recom = dataframe[dataframe['order_status'] == 'delivered'][['customer_unique_id', 'product_id', 'review_score']]
df_recom.head()

,customer_unique_id,product_id,review_score
0,861eff4711a542e4b93843c6dd7febb0,a9516a079e37a9c9c36b9b78b10169e8,4
1,290c77bc529b7ac935b93aa66c333dc3,4aa6014eceb682077f9dc4bffebc05b0,5
2,060e732b5b29e8181a18229c7b0b2b5e,bd07b66896d6f1494f5b86251848ced7,5
3,259dac757896d24d7702b9acbbff3f3c,a5647c44af977b148e0a3a4751a09e2e,5
4,345ecd01c38d18a9036ed96c73b8d066,9391a573abe00141c56e38d84d7d5b3b,5


In [20]:
#Verificando valores vazios
df_recom.isna().value_counts()

user_id  product_id  rating
False    False       False     114859
Name: count, dtype: int64

In [19]:
df_recom.rename(columns={
    'customer_unique_id':'user_id',
    'product_id':'product_id',
    'review_score':'rating'
}, inplace=True)

In [21]:
# Configuração do dataset para o Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_recom, reader)

**Configurando Melhores Carecterísticas do Modelo**

Hiperparâmetros principais do SVD
1. n_factors → Número de fatores latentes
Define a dimensão dos vetores de usuários e itens que o modelo aprende.

Em termos simples, representa quantas "características ocultas" ele tenta identificar (ex: gênero, qualidade, popularidade...).

Valor	Efeito
Pequeno (10–30)	Modelo mais simples, menos chance de overfitting.
Alto (100+)	Modelo mais expressivo, mas pode overfitar com poucos dados.
➡️ É como o número de neurônios escondidos em uma rede neural.

2. lr_all → Learning rate (taxa de aprendizado)
Controla o tamanho dos passos do algoritmo durante o treinamento.

Usado no algoritmo de descida do gradiente estocástico.

Valor	Efeito
Baixo (0.002)	Treinamento mais lento, mas mais estável.
Alto (0.01)	Treinamento mais rápido, mas pode divergir ou oscilar.
➡️ É quanto o modelo "ajusta" os pesos a cada iteração.

3. reg_all → Regularização (penalização)
Controla o quão fortemente o modelo penaliza pesos muito altos.

Ajuda a evitar overfitting (quando o modelo aprende muito bem os dados de treino, mas falha nos novos).

Valor	Efeito
Baixo (0.02)	Pouca penalização — pode overfitar.
Alto (0.2)	Alta penalização — pode subestimar relações reais.
➡️ É como pedir ao modelo para ser humilde e não confiar demais nos dados de treino.

In [ ]:
param_grid = {
    'n_factors': [10, 20, 30, 50, 100],
    'lr_all': [0.002, 0.005, 0.008, 0.01],
    'reg_all': [0.02, 0.1, 0.15, 0.2]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, joblib_verbose=1)
gs.fit(data)


In [34]:
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.02}
RMSE Score: 1.1572

MAE {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.02}
MAE Score: 0.8706


**Treino, Teste e Validação**

In [35]:
# Divisão dos dados
trainset, testset = train_test_split(data, test_size=0.25)

# Modelo com os melhores parâmetros
best_model = gs.best_estimator['rmse']
best_model.fit(trainset)

# Avaliação no conjunto de teste
predictions = best_model.test(testset)
print("RMSE no teste:", accuracy.rmse(predictions))
print("MAE no teste:", accuracy.mae(predictions))

RMSE: 1.1570
RMSE no teste: 1.1570158552927186
MAE:  0.8713
MAE no teste: 0.8713140533733064


In [41]:
def precision_recall_f1_at_k(predictions, k=3, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
    precisions, recalls, f1s = [], [], []
    for est_true in user_est_true.values():
        est_true.sort(key=lambda x: x[0], reverse=True)
        top_k = est_true[:k]
        tp = sum((true_r >= threshold) for _, true_r in top_k)
        fp = k - tp
        fn = sum((true_r >= threshold) for _, true_r in est_true[k:])
        
        precision = tp / (tp + fp) if (tp + fp) else 0
        recall = tp / (tp + fn) if (tp + fn) else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    
    return {
        'Precision': sum(precisions) / len(precisions),
        'Recall': sum(recalls) / len(recalls),
        'F1': sum(f1s) / len(f1s)
    }


In [43]:
metrics = precision_recall_f1_at_k(predictions, k=3)
print("Métricas Top-N:", metrics)

Métricas Top-N: {'Precision': 0.2748141205872361, 'Recall': 0.7777540095019009, 'F1': 0.401836041496774}


**Treino com todos os dados**

In [ ]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

In [ ]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

# Recomendação para usuário 1: itens que ele ainda não avaliou
user_id = "345ecd01c38d18a9036ed96c73b8d066"
all_items = set(iid for (_, iid, _) in trainset_full.all_ratings())
rated_items = set(j for (j, _) in trainset_full.ur[trainset_full.to_inner_uid(user_id)])
unseen_items = all_items - rated_items

# Predizer e ordenar por melhor nota prevista
recommendations = [
    (iid, best_model.predict(user_id, iid).est) for iid in unseen_items
]
recommendations.sort(key=lambda x: x[1], reverse=True)

# Top 3 recomendações
print(f"Top 3 recomendações para o usuário {user_id}:")
for iid, score in recommendations[:3]:
    print(f"Item {iid} - Nota prevista: {score:.2f}")


Top 3 recomendações para o usuário 345ecd01c38d18a9036ed96c73b8d066:
Item 0 - Nota prevista: 4.29
Item 1 - Nota prevista: 4.29
Item 2 - Nota prevista: 4.29


In [ ]:
# Função para recomendar produtos a um usuário específico
def recomendar_produtos(user_id, df, model, n_recommendations=3):
    """Gera recomendações de produtos para um usuário."""
    
    # Obtém todos os produtos únicos
    all_products = df['product_id'].unique()
    
    # Obtém produtos já avaliados pelo usuário
    rated_products = df[df['user_id'] == user_id]['product_id'].values
    
    # Filtra apenas os produtos que o usuário ainda não avaliou
    products_to_predict = [p for p in all_products if p not in rated_products]
    
    # Faz previsões para esses produtos
    predictions = [(p, model.predict(user_id, p).est) for p in products_to_predict]
    
    # Ordena por nota prevista (maior para menor)
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    return predictions[:n_recommendations]

# Exemplo: Recomendação para o usuário 1
user_id = 1
recommendations = recomendar_produtos(user_id, df, model)

print(f"\nRecomendações para o usuário {user_id}:")
for product, rating in recommendations:
    print(f"Produto {product} - Nota prevista: {rating:.2f}")

# Recomendação de Produtos

**Separação dos dados para recomendação**

In [78]:
df_ultima_compra = dataframe.groupby("customer_unique_id").agg({'order_purchase_timestamp':max})
df_ultima_compra.reset_index(inplace=True)

C:\Users\Computador\AppData\Local\Temp\ipykernel_4784\2487374714.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ultima_compra = dataframe.groupby("customer_unique_id").agg({'order_purchase_timestamp':max})


In [79]:
df_ultima_compra['order_purchase_timestamp'] = pd.to_datetime(df_ultima_compra['order_purchase_timestamp'])
data_referencia = pd.to_datetime('2018-09-10')
df_ultima_compra['dias_desde_da_ultima_compra'] = (data_referencia - df_ultima_compra['order_purchase_timestamp']).dt.days

In [ ]:
df_30_90_dias = df_ultima_compra[(df_ultima_compra['dias_desde_da_ultima_compra'] >= 30) & (df_ultima_compra['dias_desde_da_ultima_compra'] <= 90)]['customer_unique_id']
df_30_90_dias = df_30_90_dias.to_list()